# Splines aproximation for HUC Problem

## Config

### Ampl API configuration

In [1]:
# # Install dependencies
# %pip install -q --upgrade pip
# %pip install -q --upgrade amplpy ipywidgets sympy
# !python3.10 -m amplpy.modules uninstall gurobi
# !python3.10 -m amplpy.modules install cplex
# !python3.10 -m amplpy.modules installed

import sys
sys.path.append('/usr/local/ilog-2211/cplex/python/3.9/x86-64_linux')

import cplex

In [ ]:
# Imports and AMPL API configuration
from typing import List
import pandas as pd
import amplpy
from amplpy import AMPL, ampl_notebook
from IPython.display import display, Math, Latex
import matplotlib.pyplot as plt
import numpy as np

# Uncomment this part if it's the first run
# ampl = ampl_notebook(
#     modules=["gurobi", "coin"],  # modules to install
#     # license_uuid="default",  # license to use
# )

ampl = ampl_notebook()
ampl.set_option("display_precision", 17)

### Setting file names

In [ ]:
# Define the folder path where the files are located
problem_name = "HUC_scaled"
folder_path = "data/HUC"
folder_path_spline = "data/HUC/spline"

# Define the name of the temporary .lp file
new_lp="tmp_1.lp"

In [4]:
import os
from ipywidgets import interact, widgets

# List all files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
if ".DS_Store" in file_list:
    file_list.remove(".DS_Store")

# Function to load and display the selected file
def load_file(selected_file):
    global file_data
    file_data = os.path.join(folder_path, selected_file)
    print(f"Loading file: {selected_file}")

# Create an interactive dropdown for file selection
interact(load_file, selected_file=widgets.Dropdown(options=file_list, description='Select File:'));

interactive(children=(Dropdown(description='Select File:', options=('HUC_T=12.dat',), value='HUC_T=12.dat'), O…

In [5]:
# List all files in the folder
file_list = sorted([f for f in os.listdir(folder_path_spline) if os.path.isfile(os.path.join(folder_path_spline, f))])
if ".DS_Store" in file_list:
    file_list.remove(".DS_Store")

# Function to load and display the selected file
def load_file(selected_file):
    global file_dataSplines
    file_dataSplines = os.path.join(folder_path_spline, selected_file)
    print(f"Loading file: {selected_file}")

# Create an interactive dropdown for file selection
interact(load_file, selected_file=widgets.Dropdown(options=file_list, description='Select File:'));

interactive(children=(Dropdown(description='Select File:', options=('HUC_ndata-proc.txt', 'HUC_ndata_435_bound…

## Run original model

In [ ]:
solver = "gurobi"
time_limit = 600

nome_arquivo = os.path.basename(file_data)
nome_arquivo, _ = os.path.splitext(nome_arquivo)
out_log = f"out/log_file_{solver}_{nome_arquivo}_original.log"
out_sol = f"sol/sol_file_{solver}_{nome_arquivo}_original.sol"

print(out_log)
print(out_sol)

out/log_file_scip_HUC_T=12_original.log
sol/sol_file_scip_HUC_T=12_original.sol


#### Solving the original model

In [ ]:
ampl.reset()
ampl.read("HUC_scaled.mod")
ampl.readData(file_data)

ampl.option["solver"] = solver
ampl.option["gurobi_options"] = "timelim=600 outlev=1 tech:threads=1"
ampl.option["couenne_options"] = "outlev=2"
ampl.option["baron_options"] = f'maxtime=600 outlev=2'
ampl.option["bonmin_options"] = f'bonmin.time_limit=600 outlev=2'

with open("couenne.opt", 'w') as arquivo:
    arquivo.write(f"time_limit {time_limit}")

ampl.set_option("show_stats", 1)
ampl.set_option("presolve", 0)
ampl.set_option("log_file", out_log)
ampl.solve()
ampl.display("_varname, _var")
ampl.display("_obj[1]")
ampl.eval("display _solve_elapsed_time;")

ampl.set_option("log_file", "")


120 variables:
	60 binary variables
	36 nonlinear variables
	24 linear variables
169 constraints; 464 nonzeros
	12 nonlinear constraints
	157 linear constraints
	37 equality constraints
	132 inequality constraints
1 linear objective; 36 nonzeros.

scip: error while loading shared libraries: libreadline.so.6: cannot open shared object file: No such file or directory
exit value 127
<BREAK>
:        _varname     _var    :=
1     'q[1,1]'          0
2     'q[1,2]'          0
3     'q[1,3]'          0
4     'q[1,4]'          0
5     'q[1,5]'          0
6     'q[1,6]'          0
7     'q[1,7]'          0
8     'q[1,8]'          0
9     'q[1,9]'          0
10    'q[1,10]'         0
11    'q[1,11]'         0
12    'q[1,12]'         0
13    'v[1]'            0
14    'v[2]'            0
15    'v[3]'            0
16    'v[4]'            0
17    'v[5]'            0
18    'v[6]'            0
19    'v[7]'            0
20    'v[8]'            0
21    'v[9]'            0
22    'v[10]'           0
23 

## Splines

In [ ]:
# ampl = AMPL()
param_bar_n = ampl.param["bar_n"].value()
param_bar_t = ampl.param["bar_t"].value()
param_bar_k = param_bar_n * param_bar_t

print(param_bar_n, param_bar_t, param_bar_k)

In [ ]:
filepath = file_dataSplines
output_path = "data/spline/HUC_ndata-proc.txt"

with open(filepath, 'r') as f:
    content = f.read()

raw_params = content.split(';')

with open(output_path, 'w') as out:
    for param in raw_params:
        param = param.strip()
        if not param:
            continue

        proc = False

        if "**K" in param:
            proc = True
            print(param.replace('**K', str(param_bar_k)), ";\n\n", end='', file=out)

        elif "**k" in param:
            proc = True
            param2 = param.split(":=")

            if len(param2) == 2:
                print(param2[0], ":=", end='', file=out)
                for i in range(1, param_bar_k + 1):
                    print(param2[1].replace('**k', str(i)), end='', file=out)
                print(";\n\n", end='', file=out)

            elif len(param2) > 2:
                param2 = param.split("[")
                print(param2[0], end='', file=out)
                for i in range(1, param_bar_k + 1):
                    print("[", param2[1].replace('**k', str(i)), end='', file=out)
                for i in range(1, param_bar_k + 1):
                    print("\n\n[", param2[2].replace('**k', str(i)), end='', file=out)
                print(";\n\n", end='', file=out)

        elif "**j" in param and "**t" in param:
            proc = True
            count = 1
            param2 = param.split(":=")

            if len(param2) == 2:
                print(param2[0], ":=", end='', file=out)
                for i in range(1, param_bar_n + 1):
                    for j in range(1, param_bar_t + 1):
                        print(
                            param2[1]
                            .replace('**j', str(i))
                            .replace('**t', str(j))
                            .replace('**c', str(count)),
                            end='', file=out
                        )
                        count += 1
                print(";\n\n", end='', file=out)

        if not proc:
            print(param, ";\n\n", end='', file=out)

#### Defining and loading the extra data for splines

Each spline can be defined as follows:  

$Y_i = \sum_{j=1}^{\bar{N}_i} \sigma_{ij}(X_{ij}) + \alpha_{i} , \quad \forall i \in \{1, \dots, N\}$

where:

- $ Y_i $ : A dependent variable from the original formulation.
- $ X_{ij} $ : An independent variable from the original formulation.
- $ \sigma_{ij}(x) $ : A piecewise function.
- $ \alpha_{i} $ : A constant term
- $ N $ : The number of splines.
- $ \bar{N}_i $ : The number of piecewise functions in each spline.

To load the spline data, we need to define additional parameters.

In [ ]:
%%ampl_eval
# defining extra parameters for splines

# Number of splines:
param splines_n > 0;

# Number of sigma piecewise functions for each spline:
param splines_nVariables {1 .. splines_n} > 0;

# Dependent variable name for each spline:
param splines_variablesY {1 .. splines_n} symbolic;

# Sigma variables names (wich will receive the piecewise function value) for each spline:						  
param splines_variablesSigma {i in {1 .. splines_n}, 
						  1 .. splines_nVariables[i]} symbolic;
						  
# Independent variable name for each sigma piecewise function:
param splines_variablesX {i in {1 .. splines_n}, 
						  1 .. splines_nVariables[i]} symbolic;

# Number of intervals for each sigma piecewise function in each spline:
param splines_nIntervals {i in {1 .. splines_n}, 
						  1 .. splines_nVariables[i]} > 0;

# Degree of the polynomial for each sigma:
param splines_nDegree {i in {1 .. splines_n}, 
					   1 .. splines_nVariables[i]} > 0;

# Alpha parameters for each spline:
param splines_alpha {1 .. splines_n};

# Lower bound values for each interval in the piecewise function of each spline:
param splines_l {i in {1 .. splines_n},
				j in {1 .. splines_nVariables[i]},
				1 .. splines_nIntervals[i,j]+1};

# Spline function coefficients:
param splines	{i in {1 .. splines_n},
				 j in {1 .. splines_nVariables[i]}, 
				 1 .. splines_nIntervals[i,j], 
				 1 .. splines_nDegree[i,j]+1 };
  

#### Loading data

The extra data is loaded:

In [ ]:
ampl.readData(output_path)

Loading it into python variables:

In [ ]:
splines_n = ampl.param["splines_n"].value()
splines_nVariables = ampl.param["splines_nVariables"].to_dict()
splines_nIntervals = ampl.param["splines_nIntervals"].to_dict()
splines_variablesY = ampl.param["splines_variablesY"].to_dict()
splines_variablesSigma = ampl.param["splines_variablesSigma"].to_dict()
splines_variablesX = ampl.param["splines_variablesX"].to_dict()
splines_nDegree = ampl.param["splines_nDegree"].to_dict()
splines_alpha = ampl.param["splines_alpha"].to_dict()
splines_l = ampl.param["splines_l"].to_dict()
splines = ampl.param["splines"].to_dict()

#### Extra variables and constraints

The extra variables and constraints are necessary and will be automatically detected using the code below:

In [ ]:
statement = ""

# Sigma variables
for var in set(splines_variablesSigma.values()) :
    statement += f'var {var};\n'

# Plus operation variables and constraints
for var in set(splines_variablesX.values()) :
    if (var.find("_plus_") > 0) :
        vars = var.split("_plus_")
        statement += f'var {var};\n'
        statement += f'subject to splines_1_sum_{var}: {var} = {vars[0]} + {vars[1]};\n'

# Splines definition
for i in range(1,splines_n+1) :
    statement += f'subject to splines_1_definition_{i}: {splines_variablesY[i]} <= + P_minus[1]*u[1,{i}] + splines_alpha[{i}]'
    for j in range(1,splines_nVariables[i]+1) :
        statement += f' + {splines_variablesSigma[i,j]}'
    statement += ';\n'

print(statement)


In [ ]:
ampl.eval(statement)

For each spline $i$, and for each piecewise function $j$, we have:

$\sigma_{ij}(X_{i,j}) = \sum_{k=1}^{\hat{N_{i,j}}} \sigma''_{i,j,k}$, where $\hat{N_{i,j}}$ is the number of the intervals in the piecewise function $j$.

Define the binary variables:

$y_{i,j,k} \in \{0,1\}$

Charging variables:

$0 \leq x_{i,j,k} \leq l_{i,j,k+1} - l_{i,j,k}$

Sigma variables:

$\sigma''_{i,j,k} \text{ is free}$

Charging constraint:

$X_{i,j} = \sum_{k=1}^{I_{i,j}} \left( y_{i,j,k} \cdot l_{i,j,k} + x_{i,j,k} \right)$

Multiple choice constraint:

$\sum_{k=1}^{I_{i,j}} y_{i,j,k} = 1$

Interval selection constraint:

$x_{i,j,k} \leq (l_{i,j,k+1} - l_{i,j,k}) \cdot y_{i,j,k}$

Calculating the polynomial function for each interval:

$\sigma''_{i,j,k} = \sum_{d=1}^{D_{i,j}+1} c_{i,j,k,d} \cdot \left( (l_{i,j,k} + x_{i,j,k}) - l_{i,j,k} \right)^{D_{i,j}+1-d} $
$- \left( \sum_{d=1}^{D_{i,j}+1} c_{i,j,k,d} \cdot \left( (l_{i,j,k}) - l_{i,j,k} \right)^{D_{i,j}+1-d} \right) \cdot (1 - y_{i,j,k}) $

Setting the sigma variable:

$\sum_{k=1}^{I_{i,j}} \sigma''_{i,j,k} = \sigma_{i,j}$

Temos um problema quando sum y = 0

In [ ]:
for i in range(1, splines_n+1) :
    for j in range(1, splines_nVariables[i]+1) :
        # Declaring variable
        if j == 1 : # v
          ampl.eval (f'var splines_nonselectslack_{i}_{j} >= 0 <= Vub/1000;')
          ampl.eval (f's.t. splines_2_nonselectslack_{i}_{j} : splines_nonselectslack_{i}_{j} <= (Vub/1000)*(1-g[1,{i}]);')

        if j == 2 : # q
          ampl.eval (f'var splines_nonselectslack_{i}_{j} >= Q_minus[1] <= Qub[1];')
          ampl.eval (f's.t. splines_2_nonselectslack_lb1_{i}_{j} : splines_nonselectslack_{i}_{j} <= Qub[1]*(1-g[1,{i}]) + Q_minus[1]*(u[1,{i}]);')
          ampl.eval (f's.t. splines_2_nonselectslack_ub1_{i}_{j} : splines_nonselectslack_{i}_{j} >= Q_minus[1]*(u[1,{i}]);')

        ampl.eval (f'var splines_select_sigma_{i}_{j}{{1 .. splines_nIntervals[{i},{j}]}} binary;')
        ampl.eval (f'var splines_charge_sigma_{i}_{j}{{k in 1 .. splines_nIntervals[{i},{j}]}} >= 0 <= splines_l[{i},{j},k+1]-splines_l[{i},{j},k];')
        ampl.eval (f'var splines_result_sigma_{i}_{j}{{k in 1 .. splines_nIntervals[{i},{j}]}};')

        # Charging the variable
        ampl.eval(f"s.t. splines_2_cons_charge_sigma_{i}_{j} : {splines_variablesX[i,j]} == splines_nonselectslack_{i}_{j} + sum {{k in 1 .. splines_nIntervals[{i},{j}]}} (splines_select_sigma_{i}_{j}[k]*splines_l[{i},{j},k]  + splines_charge_sigma_{i}_{j}[k]);")

        # Multiple choice for binary variables
        ampl.eval (f's.t. splines_2_cons_mc_sigma_{i}_{j} : sum{{k in 1 .. splines_nIntervals[{i},{j}]}} splines_select_sigma_{i}_{j}[k] == g[1,{i}];')

        # Select the interval
        ampl.eval (f's.t. splines_2_cons_charge_select_sigma_{i}_{j} {{k in 1 .. splines_nIntervals[{i},{j}]}} : splines_charge_sigma_{i}_{j}[k] <= (splines_l[{i},{j},k+1]-splines_l[{i},{j},k])*splines_select_sigma_{i}_{j}[k];')

        # Calculate the correct function for the piecewise interval
        ampl.eval (f"""s.t. splines_2_cons_poly_sigma_{i}_{j} {{k in 1 .. splines_nIntervals[{i},{j}]}} : splines_result_sigma_{i}_{j}[k] == 
                   (sum{{d in {{1 .. splines_nDegree[{i},{j}]+1}}}} splines[{i},{j},k,d]*((splines_l[{i},{j},k] + splines_charge_sigma_{i}_{j}[k]) - splines_l[{i},{j},k])^(splines_nDegree[{i},{j}]+1 - d)) 
                 - (sum{{d in {{1 .. splines_nDegree[{i},{j}]+1}}}} splines[{i},{j},k,d]*((splines_l[{i},{j},k]) - splines_l[{i},{j},k])^(splines_nDegree[{i},{j}]+1 - d))*(1-splines_select_sigma_{i}_{j}[k]); """)
        
        # Setting the sigma variable
        ampl.eval (f's.t. splines_2_cons_set_sigma_{i}_{j} : sum{{i in 1 .. splines_nIntervals[{i},{j}]}} splines_result_sigma_{i}_{j}[i] == {splines_variablesSigma[i,j]};')


In [ ]:
# ampl.export_model(filename=f'mods/ex{problem_name}_splines.mod')

In [ ]:
def getConstraintLists() :
    constraints = {}
    constraints_drop = []
    constraints_splines_1 = []
    constraints_splines_2 = []
    for i,con in enumerate(ampl.getConstraints()) :
        constraints[i+1] = con[0]
        if(con[0][-6:] == "_drop_") :
            constraints_drop.append(i+1)
        if(con[0][:10] == "splines_1_") :
            constraints_splines_1.append(i+1)
        if(con[0][:10] == "splines_2_") :
            constraints_splines_2.append(i+1)
    return constraints, constraints_drop, constraints_splines_1, constraints_splines_2

def dropConstraints(constraints, list = []) :
    for i in list :
        ampl.getConstraint(constraints[i]).drop()

def restoreConstraints(constraints, list = []) :
    for i in list :
        ampl.getConstraint(constraints[i]).restore()

def setOriginalModel() :
    constraints, constraints_drop, constraints_splines_1, constraints_splines_2 = getConstraintLists()
    dropConstraints(constraints, constraints_splines_1)
    dropConstraints(constraints, constraints_splines_2)
    restoreConstraints(constraints, constraints_drop)

def setSplines_1() :
    constraints, constraints_drop, constraints_splines_1, constraints_splines_2 = getConstraintLists()
    restoreConstraints(constraints, constraints_splines_1)
    dropConstraints(constraints, constraints_splines_2)
    dropConstraints(constraints, constraints_drop)

def setSplines_2() :
    constraints, constraints_drop, constraints_splines_1, constraints_splines_2 = getConstraintLists()
    restoreConstraints(constraints, constraints_splines_1)
    restoreConstraints(constraints, constraints_splines_2)
    dropConstraints(constraints, constraints_drop)

constraints, constraints_drop, constraints_splines_1, constraints_splines_2 = getConstraintLists()
print(constraints)
print(constraints_drop)
print(constraints_splines_1)
print(constraints_splines_2)

#### Exploring the data

The two functions below return the function of each polynomial:
- The first function returns it in a LaTeX format so that it can be displayed on the screen.
- The second function returns it in Python and is compiled at runtime, allowing it to be used when needed.


In [ ]:
import types 

def get_function_latex(list_sp, list_l) -> str:
    degree = int((len(list_sp)-1))
    expression = ""
    for k,i in enumerate(list_sp) :
        expression += "%+.10f" % (i)
        if ( (degree - k) > 0 ) : 
            expression += "(x - %.10f)" % (list_l)
            if ( (degree - k) > 1 ) : 
                expression += "^%d" % (degree-k)
    return expression

def get_function_python(list_sp, list_l):
    degree = int((len(list_sp)-1))
    expression = ""
    for k,i in enumerate(list_sp) :
        expression += "%+.10f" % (i)
        if ( (degree - k) > 0 ) : 
            expression += "*(x - %.10f)" % (list_l)
            if ( (degree - k) > 1 ) : 
                expression += "**%d" % (degree-k)

    return types.FunctionType(compile("def func(x) : \n\treturn " + expression, '<string>', 'exec').co_consts[0], globals())

list_funv = []
for k in range(1, splines_n + 1):
    list_func_var = []
    for l in range(1, splines_nVariables[k] + 1):
        list_func_spline = []
        for i in range(1, splines_nIntervals[k,l] + 1):
            list_sp = [splines[k,l,i,j] for j in range(1,(splines_nDegree[k,l]+1)+1)]
            list_func_spline.append(get_function_python(list_sp, splines_l[k,l,i]))
        list_func_var.append(list_func_spline)
    list_funv.append(list_func_var)

#### Showing spline functions imported from the data file

In [ ]:
for k in range(1, splines_n + 1):
    variables = [f"\\sigma_{{{k},{l}}}(" + splines_variablesX[k,l].replace("_plus_", "+") + ") + " for l in range(1, splines_nVariables[k] + 1)]
    variables = " ".join(variables)
    expression = f'{splines_variablesY[k]} = {variables} {splines_alpha[k]}'
    display(Math(expression))

    for l in range(1, splines_nVariables[k] + 1):
        variableX = splines_variablesX[k,l].replace("_plus_", "+")
        expression = f'\sigma_{{{k},{l}}}({variableX}) = \\begin{{cases}}'

        for i in range(1, splines_nIntervals[k,l] + 1):
            list_sp = [splines[k,l,i,j] for j in range(1,(splines_nDegree[k,l]+1)+1)]
            expression += f'{get_function_latex(list_sp, splines_l[k,l,i])}, & {splines_l[k,l,i]} \leq {variableX} < {splines_l[k,l,i+1]} \\\\'
            list_func_spline.append(get_function_python(list_sp, splines_l[k,l,i]))

        expression += f'\\end{{cases}}'
        display(Math(expression))

#### Spline piecewise function

The value of each sigma can be calculated using the folowing Python function:

In [ ]:
def f_spiline_sigma(k : int,l : int, x, tolerance = 1e-6 ):
    
    i=1
    cond_first = [(x >= splines_l[k, l, i] - tolerance) & (x <= splines_l[k, l, i + 1])]
    i=splines_nIntervals[k,l]
    cond_last = [(x >= splines_l[k, l, i]) & (x <= splines_l[k, l, i + 1] + tolerance)]

    # Conditions for piecewise functions
    conditions_x = [
        (x >= splines_l[k,l, i]) & (x <= splines_l[k,l, i + 1]) 
        for i in range(2, splines_nIntervals[k,l] + 0)  # 1-based indexing
    ]

    conditions_x = cond_first + conditions_x + cond_last

    # Avoid late binding issue by using a lambda wrapper
    functions_x = [lambda x, i=i: list_funv[k-1][l-1][i](x) for i in range(splines_nIntervals[k,l])]

    # Piecewise function for sigma_1
    sigma_1 = np.piecewise(x, conditions_x, functions_x)

    return sigma_1  # Return results for all k

## Solving Splines with Gurobi

In [ ]:
solver = "couenne"
time_limit = 600

nome_arquivo = os.path.basename(file_dataSplines)
nome_arquivo, _ = os.path.splitext(nome_arquivo)
out_log = f"out/log_file_{solver}_{nome_arquivo}.log"
out_sol = f"sol/sol_file_{solver}_{nome_arquivo}.sol"

print(out_log)
print(out_sol)

In [ ]:
setSplines_2()

ampl.option["solver"] = solver
ampl.option["gurobi_options"] = "timelim=600 outlev=1 tech:threads=1"
ampl.option["couenne_options"] = "outlev=2"
ampl.option["baron_options"] = f'maxtime=600 outlev=2'
ampl.option["bonmin_options"] = f'bonmin.time_limit=600 outlev=2'

with open("couenne.opt", 'w') as arquivo:
    arquivo.write(f"time_limit {time_limit}")


ampl.set_option("log_file", out_log)
ampl.solve()
ampl.display("_varname, _var")
ampl.display("_obj[1]")
ampl.eval("display _solve_elapsed_time;")

ampl.set_option("log_file", "")

In [ ]:
%%ampl_eval
let {j in J, t in T} p[j,t] := 0;
let {j in J, t in T : g[j,t] >= 0.5} p[j,t] := 9.81/1000 * q[j,t] * sum{h in 0..6} ( L[h]*q[j,t]^h * (sum{k in 0..6} (K[k]*(1000*v[t])^k) - Llb - R0*q[j,t]^2 ));
let {j in J, t in T : u[j,t] >= 0.5} p[j,t] := P_minus[j];

In [ ]:
ampl.set_option("log_file", out_log)
ampl.display("_obj[1]");
ampl.set_option("log_file", "")

In [ ]:
def fix_integral_variables() :
    for name, var in ampl.getVariables() :
        output = var.__str__()

        if "integer" in output.lower() or "binary" in output.lower():
            var.fix()

def unfix_variables() :
    for name, var in ampl.getVariables() :
        var.unfix()

In [ ]:
fix_integral_variables()
setOriginalModel()

ampl.option["solver"] = "ipopt"
ampl.solve()
ampl.display("_varname, _var")

ampl.set_option("log_file", out_log)
ampl.display("_obj[1]");
ampl.set_option("log_file", "")

unfix_variables()

1 - _obj[1] = 14519.363208013572

2 - _obj[1] = 14519.363208013834

3 - _obj[1] = 14519.363208014631

4 - _obj[1] = 14522.984376768505

5 - _obj[1] = 14537.25685689077

## Solving Splines with SCMINLP

### Creating the LP file

In [ ]:
setSplines_1() 

In [ ]:
%%ampl_eval
# running the converter

option presolve 0;
option substout 0;
option auxfiles 'cr';

option solver gurobiasl;
option gurobi_options 'writeprob=tmp.lp timelim=0 outlev=0 presolve=0';

write gtmp;
solve ;

In [ ]:
setOriginalModel()

#### Formating variable names of the .lp file

The format of the .lp file exported from Gurobi is not compatible with the CPLEX library. We need to adapt the name of the variables, for example:

* ``q[1,1]`` $\rightarrow$ ``q_1_1_``



In [ ]:
!sed 's/\[/_/g; s/\]/_/g; s/\,/_/g' tmp.lp > $new_lp

### Preparing the data for SCMINLP

#### Creating the SplinesModel

In [ ]:
import io
import sys

class Ampl_extra :
    def __init__(self, ampl, spmodel):
        self.ampl = ampl
        self.spmodel = spmodel
        self.best_solution = None
        self.current_solution = None
        self.best_objValue = None
        self.current_objValue = None
        self.solver = "ipopt"
        self.objMinimization = [i for i in ampl.getObjectives()][0][1].minimization()

    def LoadSolutionScmnlp2Ampl(self) :
        solution = self.spmodel.getSolution()

        for name, var in ampl.getVariables():
            for index, _ in var:
                if isinstance(index, tuple) and len(index) > 0:
                    index_str = "_".join(str(i) for i in index)
                    var_name = f"{name}_{index_str}_"
                elif isinstance(index, int):
                    index_str = str(index)
                    var_name = f"{name}_{index_str}_"
                else :
                    var_name = f"{name}"
                
                if var_name in solution :
                    var[index] = solution[var_name]
        return solution
    
    def setBestSolution(self):
        if self.current_objValue != None :
            if self.best_objValue == None :
                self.best_objValue = self.current_objValue
                self.best_solution = self.current_solution
            elif (self.objMinimization and self.current_objValue < self.best_objValue) or \
               (self.objMinimization == False and self.current_objValue > self.best_objValue) :
                self.best_objValue = self.current_objValue
                self.best_solution = self.current_solution

    def calculate_original_function(self) : 
        self.LoadSolutionScmnlp2Ampl()
        self.ampl.read("HUC_setOriginalFunc.run")
        print(f'Original function objective      :  ', end="")
        for obj, index in ampl.getObjectives() :
            print(index.value())
    
    def solveIPOPT(self) :
        # self.LoadSolutionScmnlp2Ampl()
        self.fix_integral_variables()
        self.ampl.solve(solver=self.solver, verbose=False)
        if self.ampl.solve_result == "solved":
            self.current_objValue = self.ampl.getCurrentObjective().value()
            self.current_solution = self.ampl.get_solution()
        else :
            self.current_objValue = None
            self.current_solution = None
        self.setBestSolution()
        self.displayObjValue()
        self.unfix_variables()
    
    def displayObjValue(self) :
        print(f'Local optim value original obj   :  {self.current_objValue}')
        print(f'Best value original model        :  {self.best_objValue}')

    def fix_integral_variables(self) :
        for name, var in self.ampl.getVariables() :
            output = var.__str__()

            if "integer" in output.lower() or "binary" in output.lower():
                var.fix()

    def unfix_variables(self) :
        for name, var in self.ampl.getVariables() :
            var.unfix()

    def display_vars(self) : 
        for name, var in self.ampl.getVariables():
            for index, _ in var:
                if isinstance(index, tuple) and len(index) > 0:
                    index_str = "_".join(str(i) for i in index)
                    var_name = f"{name}_{index_str}_"
                elif isinstance(index, int):
                    index_str = str(index)
                    var_name = f"{name}_{index_str}_"
                else :
                    var_name = f"{name}"
                print(f"{var_name} = {var[index].value()}")

In [ ]:
from SCMINLP_SP_HUC_AUTO import SplinesModel

def get_function_python_code(list_sp, list_l, var : str = "x") :
    degree = int(len(list_sp)-1)
    expression = ""
    for k,i in enumerate(list_sp) :
        expression += "%+.10f" % (i)
        if ( (degree - k) > 0 ) : 
            expression += "(%s - %.10f)" % (var,list_l)
            if ( (degree - k) > 1 ) : 
                expression += "**%d" % (degree-k)
    return expression

spmodel = SplinesModel()

for k in range(1, splines_n + 1):
    for l in range(1, splines_nVariables[k] + 1):
        VarsList = [splines_variablesY[k]]
        IntervalsList = []
        Functions = []
        independentList = []
        varY_Name = splines_variablesSigma[k,l]
        varZ_Name = VarsList[0].replace("p","g").replace('[', '_').replace(']', '_').replace(',', '_') # HUC
        varX_Name = []

        for i in range(1, splines_nIntervals[k,l] + 1):
            newVarNameX = splines_variablesX[k,l].replace('[', '_').replace(']', '_').replace(',', '_')

            IntervalsList.append([splines_l[k,l,i], splines_l[k,l,i+1]])
            list_sp = [splines[k,l,i,j] for j in range(1,(splines_nDegree[k,l]+1)+1)]
            Functions.append(get_function_python_code(list_sp, splines_l[k,l,i], newVarNameX))
            independentList.append(0.0)
            varX_Name.append(newVarNameX)

        if k == 1 :
            spmodel.addSpline(IntervalsList, VarsList, independentList, Functions, varX_Name, varY_Name, 'L', varZ_Name)
        else :
            spmodel.addSpline(IntervalsList, VarsList, independentList, Functions, varX_Name, varY_Name, 'L', varZ_Name)

#### Calculating the breakpoints

In [ ]:
spmodel.breakpoints()

In [ ]:
index_display = 4
display(spmodel.SplinesList[index_display].setBreakpoints)
display(spmodel.SplinesList[index_display].setEstimation)
display(spmodel.SplinesList[index_display].Functions)

### Solve

In [ ]:
nome_arquivo = os.path.basename(file_dataSplines)
nome_arquivo, _ = os.path.splitext(nome_arquivo)
out_log = f"out/log_file_scminlp_{nome_arquivo}.log"
out_sol = f"sol/sol_file_scminlp_{nome_arquivo}.sol"

print(out_log)
print(out_sol)

In [ ]:
ampl_extra = Ampl_extra(ampl, spmodel)

iteration = 0
added = 1
gap = 100

spmodel.TIMELIMIT = 600

import sys
from contextlib import redirect_stdout

with open(out_log, 'w') as f:
    with redirect_stdout(f):
        while ( added > 0 and iteration < 200 and gap > 1e-4 ) :
            spmodel.loadLPFile(new_lp)
            spmodel.solve(iteration)
            spmodel.fixingLocal()
            spmodel.solve_display()
            ampl_extra.calculate_original_function()
            ampl_extra.solveIPOPT()

            gap = spmodel.getGap()
            display(gap)

            added = spmodel.refinement()
            iteration += 1

        spmodel.final_display()

The global optimum of (P6_2_5) is $f(\mathbf{x}^*) = -70.75207783.$

#### Export solution

In [ ]:
with open(out_sol, 'w') as f:
    with redirect_stdout(f):
        solution = ampl_extra.LoadSolutionScmnlp2Ampl()
        for var in solution :
            print(f"let {var} := {solution[var]};")

### Evaluating Spline Approximation: Solver Output vs Real values"

In [ ]:
solution = ampl_extra.LoadSolutionScmnlp2Ampl()
ampl.display("_varname, _var")
ampl.display("_obj[1]")

### Evaluating Spline Approximation: Expected Values vs. Solver Output"

In [ ]:
table_sigma = []
table_y = []

for i in range(1, splines_n+1) :
    z = 0
    for j in range(1, splines_nVariables[i]+1) :
        sigma = f_spiline_sigma(i, j, solution[splines_variablesX[i,j]] )
        z += sigma

        table_sigma.append([splines_variablesSigma[i,j], sigma, solution[splines_variablesSigma[i,j]], abs(sigma - solution[splines_variablesSigma[i,j]])])
    z += splines_alpha[i]

    table_y.append([splines_variablesY[i], z, solution[splines_variablesY[i]], abs(z - solution[splines_variablesY[i]])])


In [ ]:
pd.DataFrame(table_y, columns=["Variable name","Real value","Value from solver", "Abs. error"])

In [ ]:
pd.DataFrame(table_sigma, columns=["Variable name","Real value","Value from solver", "Abs. error"])

In [ ]:
cpx = cplex.Cplex("out.lp")
cpx.parameters.threads.set(1)
cpx.solve()

cpx.solution.get_status_string()

In [ ]:
for value, name in zip(cpx.solution.get_values(),cpx.variables.get_names()):
    print(value, name)